<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 4: рекомендательные системы</center></h2>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import math

In [3]:
!pip install --user kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 66 bytes


Загрузим данные и проведем предобработку данных как на семинаре.

In [6]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

  0% 0.00/8.20M [00:00<?, ?B/s]
100% 8.20M/8.20M [00:00<00:00, 134MB/s]
Archive:  articles-sharing-reading-from-cit-deskdrop.zip
  inflating: articles/shared_articles.csv  
  inflating: articles/users_interactions.csv  


In [7]:
articles_df = pd.read_csv("articles/shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [8]:
interactions_df = pd.read_csv("articles/users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [9]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [10]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   "VIEW": 1.0,
   "LIKE": 2.0, 
   "BOOKMARK": 2.5, 
   "FOLLOW": 3.0,
   "COMMENT CREATED": 4.0,  
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [11]:
users_interactions_count_df = (
    interactions_df
    .groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId").size())
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[["personId"]]
print("# users with at least 5 interactions:",len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [12]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [13]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглаживаем полученный результат, взяв от него логарифм.

In [14]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"]).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])["timestamp"].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [15]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [16]:
interactions = (
    interactions_train_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = (
    interactions_test_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    "" for x in range(len(interactions.loc[pd.isnull(interactions.true_test), "true_test"]))]

interactions.head(1)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [18]:
!pip install lightfm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 310 kB 12.3 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705372 sha256=ff3c2a2ed9ca4a35d80c671366a414aa89f476d9b265d6c28672a503940c5a1b
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [19]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

## Задание 1 (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [20]:
# Ваш код здесь

unique_persons = set(interactions_full_df['contentId'])
unique_contents = set(interactions_full_df['personId'])

print("Size we need: (", len(unique_contents), ",", len(unique_persons), ")")

Size we need: ( 1140 , 2984 )


In [21]:
def sparse_matrix(data):    
    contents_all = set(interactions_train_df['contentId']) | set(interactions_test_df['contentId'])
    persons_all = set(interactions_train_df['personId']) | set(interactions_test_df['personId'])
    matrix = pd.DataFrame(0, columns=contents_all, index=persons_all)
    
    persons = data['personId'].values
    contents = data['contentId'].values
    eventStrengths = data['eventStrength'].values
    
    for i in range(len(data)):
      if (eventStrengths[i]):
        matrix.loc[persons[i], contents[i]] = eventStrengths[i]
      else:
        matrix.loc[persons[i], contents[i]] = 0
    
    return matrix

In [22]:
train = sparse_matrix(interactions_train_df)
test = sparse_matrix(interactions_test_df)

In [93]:
print(train.shape, test.shape)

(1140, 2984) (1140, 2984)


In [23]:
from scipy import sparse

data_train = sparse.coo_matrix(train.fillna(0).values)
data_test = sparse.coo_matrix(test.fillna(0).values)

## Задание 2 (1 балл)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [ ]:
# Ваш код здесь
model_lightFM = LightFM(k=10, loss='warp', random_state=666)
model_lightFM.fit(data_train, epochs=50)

In [ ]:
print("Test precision:", precision_at_k(model_lightFM, data_test, k=10).mean())

Test precision: 0.004480652


## Задание 3 (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Ваш код здесь
vectorizer = TfidfVectorizer()
feat = vectorizer.fit_transform(articles_df['text'])

In [ ]:
articles_df['text'].shape[0]

3047

In [ ]:
feat.shape

(3047, 72615)

In [ ]:
model_lightFM = LightFM(k=10, loss='warp', random_state=666)
model_lightFM.fit(data_train, epochs=50, item_features=feat)
print("Test precision:", precision_at_k(model_lightFM, data_test, item_features=feat, k=10).mean())

Test precision: 0.004378819


## Задание 4 (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к нормальной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [ ]:
# Ваш код здесь
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import re


def tokenizer(review):
    cleaned_review = re.sub("[^A-Za-z0-9]+", " ", review)
    cleaned_review = cleaned_review.lower()

    tokens = nltk.sent_tokenize(cleaned_review)

    stop_words_en = stopwords.words("english")
    stop_words_pt = stopwords.words("portuguese")

    filtered_review_en = [word for word in cleaned_review if word not in stop_words_en]
    filtered_review = [word for word in filtered_review_en if word not in stop_words_pt]

    stemmer = PorterStemmer()
    stemmer_review = [stemmer.stem(word) for word in filtered_review]

    return stemmer_review

vectorizer = TfidfVectorizer(tokenizer=tokenizer)
feat = vectorizer.fit_transform(articles_df['text'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
model_lightFM = LightFM(k=10, loss='warp', random_state=666)
model_lightFM.fit(data_train, epochs=50, item_features=feat)
print("Test precision:", precision_at_k(model_lightFM, data_test, item_features=feat, k=10).mean())

Test precision: 0.0024439918


Улучшилось ли качество предсказания? Ухудшилось, но это какой-то дьяволский random_state, до этого все улучшалось :(

## Задание 5 (2 балла)

Подберите гиперпараметры модели LightFM (`no_components` и др.) для улучшения качества модели.

In [ ]:
!pip install scikit_optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 100 kB 2.2 MB/s 


За основу следующей части кода взята статья: https://www.ethanrosenthal.com/2016/11/07/implicit-mf-part-2/

In [ ]:
from skopt import forest_minimize

def objective(params):
    # unpack
    epochs, learning_rate, no_components, alpha = params
    
    user_alpha = alpha
    item_alpha = alpha
    model = LightFM(loss='warp',
                    random_state=666,
                    learning_rate=learning_rate,
                    no_components=no_components, 
                    user_alpha=user_alpha,
                    item_alpha=item_alpha)
    model.fit(data_train, epochs=epochs, item_features=feat, num_threads=4)
    
    patks = precision_at_k(model, data_test, k=10, item_features=feat, num_threads=4)
    mapatk = np.mean(patks)
    # Make negative because we want to _minimize_ objective
    out = -mapatk
    # Handle some weird numerical shit going on
    if np.abs(out + 1) < 0.01 or out < -1.0:
        return 0.0
    else:
        return out

In [ ]:
space = [(1, 260), # epochs
         (10**-4, 1.0, 'log-uniform'), # learning_rate
         (20, 200), # no_components
         (10**-6, 10**-1, 'log-uniform'), # alpha
        ]

res_fm = forest_minimize(objective, space, n_calls=20, random_state=666, verbose=True)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 15.8140
Function value obtained: -0.0007
Current minimum: -0.0007
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 48.5418
Function value obtained: -0.0045
Current minimum: -0.0045
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 5.5828
Function value obtained: -0.0026
Current minimum: -0.0045
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 17.3065
Function value obtained: -0.0008
Current minimum: -0.0045
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 10.9958
Function value obtained: -0.0026
Current minimum: -0.0045
Iteration No: 6 start

In [ ]:
print('Maximimum p@k found: {:6.5f}'.format(-res_fm.fun))
print('Optimal parameters:')
params = ['epochs', 'learning_rate', 'no_components', 'alpha']
for (p, x_) in zip(params, res_fm.x):
    print('{}: {}'.format(p, x_))

Maximimum p@k found: 0.01303
Optimal parameters:
epochs: 44
learning_rate: 0.6275445847411844
no_components: 180
alpha: 0.00023116420340253387


In [ ]:
# Ваш код здесь
model_lightFM = LightFM(learning_rate=0.6275445847411844, 
                        loss='warp', 
                        no_components=180, 
                        user_alpha=0.00023116420340253387,
                        item_alpha=0.00023116420340253387,
                        random_state=666)
model_lightFM.fit(data_train, epochs=44, item_features=feat)
print("Test precision:", precision_at_k(model_lightFM, data_test, item_features=feat, k=10).mean())

Test precision: 0.0075356415


## Бонусное задание (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [24]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 4.2 MB 54.9 MB/s 
     |████████████████████████████████| 1.2 MB 46.9 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
     |████████████████████████████████| 596 kB 70.0 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=64f4a7632a75aac93377eb405f679b5bd64f6fa72bb0d062ae117ac8870a3e89
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# Ваш код здесь
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("average_word_embeddings_glove.6B.300d") 
embeddings = model.encode(list(articles_df['text']))
feat = sparse.coo_matrix(embeddings)

In [ ]:
model_lightFM = LightFM(learning_rate=0.6275445847411844, 
                        loss='warp', 
                        no_components=180, 
                        user_alpha=0.00023116420340253387,
                        item_alpha=0.00023116420340253387,
                        random_state=666)
model_lightFM.fit(data_train, epochs=1, item_features=feat)
print("Test precision:", precision_at_k(model_lightFM, data_test, item_features=feat, k=10).mean())

С этой библиотекой у меня все постоянно вылетало :(
А одна эпоха обучается больше 20 минут, поэтому я не успела :(